In [ ]:
import pickle
import pandas as pd
import numpy as np
import transformers

train_data = pd.read_csv('stockprice_master_NSE-TATAGLOBAL.csv')

In [ ]:
transformer = transformers.Transformer()
transformer.fit(train_data)

In [ ]:
X_train, y_train = transformer.sequence( transformer.transform(train_data))

In [ ]:
from keras_tuner import HyperParameters, GridSearch, RandomSearch
from keras.layers import Input,Dense, LSTM, Dropout, BatchNormalization as BatchNorm
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [ ]:
def model_builder(hp: HyperParameters) -> Model:
    inputs = Input(shape=(X_train[1].shape))

    lstm1 = LSTM(140,return_sequences=True)(inputs)
    
    dropout1 = Dropout(0.2)(lstm1)


    lstm2 = LSTM(units=80, return_sequences=True)(dropout1)
    
    dropout2 = Dropout(0.2)(lstm2)


    lstm3 = LSTM(units=140)(dropout2)
    dropout3 = Dropout(0.2)(lstm3)
   
    output = Dense(units=1)(dropout3)
    model = Model(inputs,output,name='stock-price-prediction')

    model.compile(optimizer=Adam(learning_rate=0.005),loss='mean_squared_error')
    return model

In [ ]:
early_stopping = EarlyStopping(monitor='loss',
                               min_delta=0.05,  # najmanja promena u odnosu na prethodnu epohu
                               patience=2,
                               # broj epoha sa promenom manjom od delta vrednosti nakon ceka trening staje
                               verbose=1,  # da prikaze zasto je trening zaustavljen
                               restore_best_weights=True)

In [ ]:
tuner = RandomSearch(model_builder,objective='loss', seed=100, max_trials=100)

In [ ]:
tuner.search(X_train,y_train,epochs = 5)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
hypermodel = tuner.hypermodel.build(best_hps)
history = hypermodel.fit(X_train, y_train, epochs=50, callbacks=[early_stopping])



In [ ]:
hypermodel.save('results/model.keras')

In [ ]:

with open('transformer', 'wb') as filepath:
    pickle.dump(transformer,filepath)